# Foundation Models para Dados Tabulares
## Comparacao: XGBoost vs TabPFN vs MITRA

Este notebook demonstra a comparacao entre modelos classicos e Foundation Models.

### Trade-off: Performance vs Interpretabilidade

| Modelo | Performance | Interpretabilidade |
|--------|-------------|-------------------|
| XGBoost + SHAP | Boa | **Excelente** |
| TabPFN/MITRA | **Excelente** | Limitada |

> **Em credit scoring, explicabilidade e requisito regulatorio (LGPD, BACEN).**

### Requisitos para TabPFN
TabPFN v6+ requer autenticacao no HuggingFace:
1. Acesse: https://huggingface.co/Prior-Labs/tabpfn_2_5
2. Aceite os termos de uso
3. Execute: `huggingface-cli login`

In [1]:
import sys
sys.path.insert(0, '../src')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

from credit_scoring.data.loader import load_german_credit
from credit_scoring.models.train import load_model

print("Setup completo!")

Setup completo!


In [2]:
X, y = load_german_credit(save_raw=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Treino: {len(X_train)} | Teste: {len(X_test)}")

Treino: 800 | Teste: 200


## XGBoost (Baseline)

In [3]:
xgb_model = load_model('../models/best_model_optuna.joblib')
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
xgb_pred = xgb_model.predict(X_test)

xgb_metrics = {
    'Model': 'XGBoost (Optuna)',
    'ROC-AUC': roc_auc_score(y_test, xgb_proba),
    'F1-Score': f1_score(y_test, xgb_pred),
    'Precision': precision_score(y_test, xgb_pred),
    'Recall': recall_score(y_test, xgb_pred),
    'Interpretable': 'Sim (SHAP)'
}
print(f"XGBoost ROC-AUC: {xgb_metrics['ROC-AUC']:.4f}")

XGBoost ROC-AUC: 0.8058


## TabPFN (Foundation Model)

**Nota**: TabPFN v6+ requer autenticacao HuggingFace.

In [4]:
tabpfn_metrics = None

try:
    from tabpfn import TabPFNClassifier
    print("TabPFN instalado. Tentando inicializar...")
    print("NOTA: TabPFN v6+ requer autenticacao no HuggingFace.")
    print("Execute: huggingface-cli login")
except ImportError:
    print("TabPFN nao instalado.")

# Descomente para testar apos autenticacao:
# X_train_p = xgb_model.named_steps['preprocessor'].transform(X_train)
# X_test_p = xgb_model.named_steps['preprocessor'].transform(X_test)
# tabpfn = TabPFNClassifier()
# tabpfn.fit(X_train_p, y_train)
# tabpfn_proba = tabpfn.predict_proba(X_test_p)[:, 1]
# print(f"TabPFN ROC-AUC: {roc_auc_score(y_test, tabpfn_proba):.4f}")

TabPFN instalado. Tentando inicializar...
NOTA: TabPFN v6+ requer autenticacao no HuggingFace.
Execute: huggingface-cli login


## Comparacao Final

In [5]:
results = [xgb_metrics]
if tabpfn_metrics:
    results.append(tabpfn_metrics)

comparison_df = pd.DataFrame(results)
print("\n" + "=" * 60)
print("COMPARACAO DE MODELOS")
print("=" * 60)
print(comparison_df.to_string(index=False))

comparison_df.to_csv('../reports/foundation_models_comparison.csv', index=False)
print("\nSalvo em: reports/foundation_models_comparison.csv")


COMPARACAO DE MODELOS
           Model  ROC-AUC  F1-Score  Precision  Recall Interpretable
XGBoost (Optuna) 0.805833  0.657534    0.55814     0.8    Sim (SHAP)

Salvo em: reports/foundation_models_comparison.csv


## Conclusao

| Cenario | Modelo Recomendado |
|---------|-------------------|
| Producao (regulado) | **XGBoost + SHAP** |
| Prototipo/PoC | TabPFN/MITRA |
| Benchmark | Comparar ambos |

**Recomendacao**: Usar XGBoost como modelo principal por causa da explicabilidade regulatoria.